---
#### 작업형 1-1. age컬럼의 3사분위 수와 1사분위 수 차이를 절대값으로 구하라 (소수점 이하 버림, 정수출력)
---

In [2]:
import numpy as np
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part4/ch4/data4-1.csv")
print(df.shape)
display(df.head())

age_quantile_1 = df['age'].quantile(0.25)
age_quantile_3 = df['age'].quantile(0.75)
print('정답 = ', int(abs(age_quantile_3 - age_quantile_1)))

(100, 8)


,id,age,city,f1,f2,f3,f4,f5
0,id01,2.0,서울,NaN,0,NaN,ENFJ,91.297791
1,id02,9.0,서울,70.0,1,NaN,ENFJ,60.339826
2,id03,27.0,서울,61.0,1,NaN,ISTJ,17.252986
3,id04,75.0,서울,NaN,2,NaN,INFP,52.667078
4,id05,24.0,서울,85.0,2,NaN,ISFJ,29.269869


정답 =  50


---
#### 작업형 1-2. 모든 reactions 중 loves와 wows를 합한 반응비율이 40% 보다 크고 50% 보다 작은 데이터를 찾고, 이 중 type 컬럼이 video인 데이터 갯수를 구하라
---

In [5]:
df = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part4/ch4/data4-2.csv")
print(df.shape)
display(df.head())

cond_ratio = (df['loves'] + df['wows']) / df['reactions']
df_2 = df[
    (cond_ratio > 0.4) & (cond_ratio < 0.5)
]
print('정답 = ', len(df_2[df_2['type'] == 'video']))

(7050, 11)


,id,type,reactions,comments,shares,likes,loves,wows,hahas,sads,angrys
0,1,video,529,512,262,432,92,3,1,1,0
1,2,photo,150,0,0,150,0,0,0,0,0
2,3,video,227,236,57,204,21,1,1,0,0
3,4,photo,111,0,0,111,0,0,0,0,0
4,5,photo,213,0,0,204,9,0,0,0,0


정답 =  90


---
#### 작업형 1-3. 2018년 1월달의 영국(United Kingdom)의 데이터 갯수는?
---

In [18]:
df = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part4/ch4/data4-3.csv")
print(df.shape)
display(df.head(2))

df['date_added'] = pd.to_datetime(df['date_added'])
#df['year'] = df['date_added'].dt.year
#df['month'] = df['date_added'].dt.month
df['YearMonth'] = df['date_added'].dt.to_period('M')

df_new = df[
    (df['YearMonth'] == '2018-01') & (df['country'] == 'United Kingdom')
]
print('정답 = ', len(df_new))

(8807, 11)


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries"


정답 =  6


---
#### 작업형 2-1. 고객의 개인정보와 행동패턴을 기반으로 고객 세분화 결과를 예측하라
- 타겟: Segmentation (고객군 1,2,3,4)
- 평가: F1 score macro
- 제출: result.csv (ID, Segmentation)
---

In [38]:
train = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part4/ch4/train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part4/ch4/test.csv")
print(train.shape, test.shape)
display(train.head(2), test.head(2))

##############  EDA  #######################################################################################
# 숫자형, 범주형 변수 혼함 & 결측치 없음
# 수치형 변수들의 분포 불균형
# train/test간 obj 변수들의 카테고리는 동일함
############################################################################################################

# 1.id삭제, 타겟분리
y = train['Segmentation']
train = train.drop(['ID', 'Segmentation'], axis=1)
test_id = test['ID']
test = test.drop('ID', axis=1)
print(train.shape, test.shape)

# 2. 수치형변수 스케일링
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
scaler = StandardScaler()
num_cols = ['Age', 'Work_Experience', 'Family_Size']
train[num_cols] = scaler.fit_transform(train[num_cols])
test[num_cols] = scaler.transform(test[num_cols])

# 3. 범주형변수 원-핫 인코딩
train = pd.get_dummies(train)
test = pd.get_dummies(test)

# 4. 검증데이터 분리
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(train, y, random_state=42)

# 5. model, eval
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
model = RandomForestClassifier(random_state=120)
model.fit(X_train, y_train)

model_pred = model.predict(X_valid)
score = f1_score(y_valid, model_pred, average='macro')
print('F1 score macro = ', score)

(6665, 11) (2154, 10)


,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,4
1,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,2


,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1
0,458989,Female,Yes,36,Yes,Engineer,0.0,Low,1.0,Cat_6
1,458994,Male,Yes,37,Yes,Healthcare,8.0,Average,4.0,Cat_6


(6665, 9) (2154, 9)
F1 score macro =  0.4773745373285647


In [39]:
# LightGBM
from lightgbm import LGBMClassifier
model2 = LGBMClassifier(random_state=120)
model2.fit(X_train, y_train)

model2_pred = model2.predict(X_valid)
score2 = f1_score(y_valid, model2_pred, average='macro')
print('f1 score macro = ', score2)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000271 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 144
[LightGBM] [Info] Number of data points in the train set: 4998, number of used features: 28
[LightGBM] [Info] Start training from score -1.414294
[LightGBM] [Info] Start training from score -1.439295
[LightGBM] [Info] Start training from score -1.352073
[LightGBM] [Info] Start training from score -1.342835
f1 score macro =  0.5095113360149297


In [37]:
#submit
Segmentation = model2.predict(test)
result = pd.DataFrame({
    'ID':test_id, 'Segmentation':Segmentation
})
result.to_csv('result.csv', index=False)